# GetURLsForCatalogJSON.ipynb

This notebook demonstrates how to read the file `catalog.json`, downloadable by clicking `Download complete table data` from https://data.black-holes.org/waveforms/catalog.html, how to get the DOI for each simulation in that file, and how to query Zenodo for its metadata for each DOI. 

This notebook writes each zenodo metadata file to a file.

NOTE: this notebook is primarily educational, to show how one can directly interact with zenodo. See the notebook `sxs_example.ipynb` for how to do this using the `sxs` python module much faster and more simply.

In [ ]:
import json
import requests
import os
import numpy as np

# Read in the file from black-holes.org/waveforms

In [ ]:
with open('catalog.json') as catalog_file:  
    catalog_json = json.load(catalog_file)

Get the DOI and name of each simulation in the catalog file.

In [ ]:
simulation_name_list = []
simulation_doi_list = []
for simulation in catalog_json:
    if (str(simulation['name']).split(':')[1] == 'BBH'):
        simulation_name_list.append(simulation['name'])
        simulation_doi_list.append(simulation['url'])
print(str(len(simulation_doi_list))+" simulations in catalog")

Here's an example of how to resolve a DOI into json containing list of files into a bibtex reference.

In [ ]:
request = requests.get(simulation_doi_list[444], headers={'accept': 'application/x-bibtex'})
request.text

# Get zenodo json file for each simulation in the catalog

This function gets Zenodo metadata, stored in json format, from a DOI.

In [ ]:
def get_zenodo_json(doi):
    request = requests.get(doi, headers={'accept': 'application/citeproc+json'})
    base_url = request.json()['URL']
    record = base_url.split('/')[-1] #gets the record number, which is the last part of a zenodo URL
    json_url = "https://zenodo.org/api/records/"+record
    request = requests.get(json_url, headers={'accept': 'application/json'})
    record_json = request.json()
    return record_json

This makes a directory and loops over all DOIs, saving each one to a file.

In [ ]:
os.mkdir("catalog_json")

In [ ]:
zenodo_json_all_simulations = {}
for i,name in enumerate(simulation_name_list):
    doi = simulation_doi_list[i]
    print(name + " -- " + doi)
    zenodo_json_all_simulations[name] = get_zenodo_json(doi)
    with open("catalog_json/"+name.replace(":","_")+".json", 'w') as outfile:
        json.dump(zenodo_json_all_simulations[name], outfile)